## imports

In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from moviepy.editor import VideoFileClip
from moviepy.video.io.VideoFileClip import VideoFileClip

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2678784/45929032 bytes (5.8%)5496832/45929032 bytes (12.0%)8355840/45929032 bytes (18.2%)11157504/45929032 bytes (24.3%)14114816/45929032 bytes (30.7%)16982016/45929032 bytes (37.0%)19660800/45929032 bytes (42.8%)22552576/45929032 bytes (49.1%)25346048/45929032 bytes (55.2%)28237824/45929032 bytes (61.5%)31006720/45929032 bytes (67.5%)34480128/45929032 bytes (75.1%)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## functions to show and plt images

In [ ]:
def show_image (image, title= "Image", cmap_type ='gray'): #define func to take the image and plot it  
  plt.imshow(image, cmap_type)
  plt.title(title)
  plt.axis("off")

In [ ]:
def plt_images(orig_image, orig_title, processed_image, processed_title, cmap='gray'):
    # Visualize undirstorsion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 70))
    ax1.set_title(orig_title, fontsize=30)
    ax1.imshow(orig_image)
    ax2.set_title(processed_title, fontsize=30)
    ax2.imshow(processed_image, cmap='gray')
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## canny fn

In [ ]:
#this fn gets canny edges for s channel of the original image and return binary image
def canny(img):
      # Convert to HLS color space to use s_channel as it will give better effect in sunny regions 
      hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
      l_channel = hls[:,:,1]  
      s_channel = hls[:,:,2]
      h_channel = hls[:,:,0]
      canny_edges = cv2.Canny(s_channel,50,150)
      canny_binary = np.zeros_like(canny_edges)
      canny_binary[(canny_edges==255 )] = 1
      return canny_binary
  

## warp and inv_warp

In [ ]:
def perspective_warp(img, 
                     dst_size=(1280,720),
                     src=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]),#[(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)] [(0.30,0.65),(0.60,0.65),(0.1,1),(1,1)]
                     dst=np.float32([(0,0), (1, 0), (0,1), (1,1)])):
    img_size = np.float32([(img.shape[1],img.shape[0])])
    src = src* img_size
    # For destination points, I'm arbitrarily choosing some points to be a nice fit for displaying our warped result again, not exact, but close enough for our purposes
    dst = dst * np.float32(dst_size)
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, dst_size,cv2.INTER_LINEAR)
    return warped

In [ ]:
def inv_perspective_warp(img, 
                     dst_size=(1280,720),
                     src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                     dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)])):#[(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)] [(0.30,0.65),(0.60,0.65),(0.1,1),(1,1)]
    img_size = np.float32([(img.shape[1],img.shape[0])])
    src = src* img_size
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = dst * np.float32(dst_size)
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, dst_size,cv2.INTER_LINEAR)
    return warped

## draw rectangle before and after warping

In [ ]:
def Draw_rectangle(warped_img):
  rect = np.dstack((warped_img, warped_img, warped_img))*255  #warped_img is binary 1 channel so we convert it to colored mage 3 channels so we can draw colored rectangl on it
  # rect = np.ones_like(Final_warp)
  rect = cv2.rectangle(rect,(70,0),(1220,720),(255,0,0),20)  #to draw rect you need top-left corner and bottom-right corner of rectangle
  # test =cv2.add(Final_warp,rect)
  #bitwiseOr = cv2.bitwise_or(rect, Final_warp)
  return rect #return rectangle or rect??????????????? rect:return the warped image with the rectangle drawing on it

In [ ]:
def img_with_rectangle(rect,after_warp):
  Rectangle = cv2.add(rect,after_warp, dtype = cv2.CV_8U)
  return Rectangle

## window sliding 

## tharwat

In [ ]:
def pixels_in_window(nonzerox,nonzeroy,center, margin, height):
        """ Return all pixel that in a specific window
        Parameters:
            center (tuple): coordinate of the center of the window
            margin (int): half width of the window
            height (int): height of the window
        Returns:
            pixelx (np.array): x coordinates of pixels that lie inside the window
            pixely (np.array): y coordinates of pixels that lie inside the window
        """
        topleft = (center[0]-margin, center[1]-height//2)
        bottomright = (center[0]+margin, center[1]+height//2)

        condx = (topleft[0] <= nonzerox) & (nonzerox <= bottomright[0])
        condy = (topleft[1] <= nonzeroy) & (nonzeroy <= bottomright[1])
        return nonzerox[condx&condy], nonzeroy[condx&condy]

## find lane pixels

In [ ]:
def find_lane_pixels(image): #image --> after warping image
    assert(len(image.shape) ==2)
    histogram = np.sum(image[image.shape[0] // 2:, :], axis=0)
    out_img = np.dstack((image, image, image)) * 255 #input image is 1 channel so convert it to 3 channel
    # out_img will be the warped image with the sliding window on it 
    midpoint = np.int(histogram.shape[0] // 2) # get midpoint of histogram
    leftx_base = np.argmax(histogram[:midpoint]) # get index of peak in the first half of histogram which will correspond to the position of left lane
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint # get index of peak in the second half of histogram which will correspond to the position of right lane

    nwindows = 9 # no of windows 
    margin = 100 # width of window /2
    minpix = 50 

    window_height = np.int(image.shape[0] // nwindows) # get window_height by dividing the image height / nwindows
    # print("image:",image)
    # nonzero(): Return : [tuple_of_arrays] Indices of elements that are non-zero
    nonzero = image.nonzero()        # to get the location of nonzero pixels in image
    # print("image.nonzero: ",nonzero)
    # [[0,5,0,0,1],
    # [0,0,0,0,0],
    # [4,7,-3,0,0]]
    nonzeroy = np.array(nonzero[0])  # to get the rows that have non zero           [0,0,2,2,2]
    nonzerox = np.array(nonzero[1])  # to get the index of each non zero element    [1,4,0,1,2]
    # print("nonzeroy: ",nonzero[0])
    # print("nonzerox: ",nonzero[1])

    # let the start point for window sliding is @ the 2 peaks of histogram
    leftx_current = leftx_base
    rightx_current = rightx_base
    y_current = image.shape[0] + window_height//2

    left_lane_inds = []
    right_lane_inds = []
    leftx=[] 
    lefty=[] 
    rightx=[] 
    righty=[]

    for window in range(nwindows):
        #first window
        # Identify window boundaries in x and y (and right and left)
        win_y_low = image.shape[0] - (window + 1) * window_height  # y of top edge of window
        win_y_high = image.shape[0] - window * window_height       # y of bottom edge of window
        win_xleft_low = leftx_current - margin                     # x of left edge of left window
        win_xleft_high = leftx_current + margin                    # x of right edge of left window
        win_xright_low = rightx_current - margin                   # x of left edge of right window
        win_xright_high = rightx_current + margin                  # x of right edge of right window

        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 4)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 4)

        # Identify the nonzero pixels in x and y within the window ###
        # good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        # #good_left_inds -> array contain indices of non zero values inside the left window region
        # good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # #good_right_inds -> array contain indices of non zero values inside the right window region
        y_current -= window_height
        center_left = (leftx_current, y_current)
        center_right = (rightx_current, y_current)
        good_left_x, good_left_y = pixels_in_window(nonzerox,nonzeroy,center_left, margin, window_height)
        good_right_x, good_right_y = pixels_in_window(nonzerox,nonzeroy,center_right, margin, window_height)
        leftx.extend(good_left_x)
        lefty.extend(good_left_y)
        rightx.extend(good_right_x)
        righty.extend(good_right_y)

        # Append these indices to the lists
        # left_lane_inds.append(good_left_inds)
        # right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_x) > minpix:
            leftx_current = np.int32(np.mean(good_left_x))
        if len(good_right_x) > minpix:
            rightx_current = np.int32(np.mean(good_right_x))


        # if len(good_left_inds) > minpix:#>
        #     leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        # if len(good_right_inds) > minpix:#>
        #     rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    # try:
    #     left_lane_inds = np.concatenate(left_lane_inds)
    #     right_lane_inds = np.concatenate(right_lane_inds)
    # except ValueError:
    #     # Avoids an error if the above is not implemented fully
    #     pass

    # # Extract left and right line pixel positions
    # leftx = nonzerox[left_lane_inds]
    # lefty = nonzeroy[left_lane_inds]
    # rightx = nonzerox[right_lane_inds]
    # righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


## fit_poly

In [ ]:
# Fit a poly to perform a directed search in well known areas
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])# Y values:[0:h-1] , number of samples = h
    # Calc both polynomials using ploty, left_fit and right_fit
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]# aY^2 +bY+c
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]# aY^2 +bY+c

    return left_fitx, right_fitx, ploty


## search around poly

In [ ]:
left_fit=[]
right_fit=[]
def search_around_poly(image):
    margin = 100

    # Grab activated pixels
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx, lefty, rightx, righty, out_img = find_lane_pixels(image)
    colored_img=np.dstack((image, image, image))
    if len(lefty) > 1500:
      global left_fit
      left_fit = np.polyfit(lefty, leftx, 2)
    if len(righty) > 1500:
      global right_fit
      right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    maxy = image.shape[0] - 1
    miny = image.shape[0] // 3
    
    if len(lefty):
        maxy = max(maxy, np.max(lefty))
        miny = min(miny, np.min(lefty))

    if len(righty):
        maxy = max(maxy, np.max(righty))
        miny = min(miny, np.min(righty))

    ploty = np.linspace(miny, maxy, image.shape[0])

    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty +left_fit[2]
    right_fitx =right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    for i, y in enumerate(ploty):
            l = int(left_fitx[i])
            r = int(right_fitx[i])
            y = int(y)
            cv2.line( colored_img, (l, y), (r, y), (0, 255, 0))
    left_curverad, right_curverad, pos=measure_curvature(left_fit,right_fit)
# ###############################################
#     window_img = np.copy(out_img)
#     filled_lanes_img = np.copy(out_img) 
#     if ((len(leftx) == 0) or (len(rightx) == 0) or (len(righty) == 0) or (len(lefty) == 0)):  
#         out_img = np.dstack((image, image, image)) * 255
#         left_curverad = 0
#         right_curverad = 0
#         center_diff =0
#     else:
#         left_fit = np.polyfit(lefty, leftx, 2)
#         right_fit = np.polyfit(righty, rightx, 2)

#         ### Set the area of search based on activated x-values ###
#         ### within the +/- margin of our polynomial function ###
#         left_lane_inds = ((nonzerox > (left_fit[0] * (nonzeroy ** 2) + left_fit[1] * nonzeroy +
#                                        left_fit[2] - margin)) & (nonzerox < (left_fit[0] * (nonzeroy ** 2) +
#                                                                              left_fit[1] * nonzeroy + left_fit[
#                                                                                  2] + margin)))
#         right_lane_inds = ((nonzerox > (right_fit[0] * (nonzeroy ** 2) + right_fit[1] * nonzeroy +
#                                         right_fit[2] - margin)) & (nonzerox < (right_fit[0] * (nonzeroy ** 2) +
#                                                                                right_fit[1] * nonzeroy + right_fit[
#                                                                                    2] + margin)))

#         # Again, extract left and right line pixel positions
#         leftx = nonzerox[left_lane_inds]
#         lefty = nonzeroy[left_lane_inds]
#         rightx = nonzerox[right_lane_inds]
#         righty = nonzeroy[right_lane_inds]

#         # Fit new polynomials
#         left_fitx, right_fitx, ploty = fit_poly(image.shape, leftx, lefty, rightx, righty)


#         ym_per_pix = 30 / 720  # meters per pixel in y dimension
#         xm_per_pix = 3 / 500  # meters per pixel in x dimension

#         # Calculate the curvature radii
#         left_fit_cr = np.polyfit(ploty * ym_per_pix, left_fitx * xm_per_pix, 2) #left_fit_circle
#         right_fit_cr = np.polyfit(ploty * ym_per_pix, right_fitx * xm_per_pix, 2)
#         y_eval = np.max(ploty)
#         left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
#             2 * left_fit_cr[0])
#         right_curverad = ((1 + (
#                     2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
#             2 * right_fit_cr[0])
                    
# #################### added to calculate center offset ##########################################
#         camera_center = (left_fitx[-1] + right_fitx[-1])/2
#         center_diff = (camera_center-image.shape[1]/2)*xm_per_pix

#         ## Visualization ##
#         # Create an image to draw on and an image to show the selection window
#         out_img = np.dstack((image, image, image)) * 255
#         # window_img = np.zeros_like(out_img)
# #######################################################################################
# # we need to draw two lanes by fitting 2 line through the points
#         # Color in left and right line pixels
#         out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
#         out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# ####################################################################################### 
#         filled_lanes_img = np.copy(out_img) 

#         # Generate and draw a poly to illustrate the lane area
#         left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
#         right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
#         points = np.hstack((left, right))
#         out_img = cv2.fillPoly(out_img, np.int_(points), (0, 255, 255))
        
#         #out img is the warped image with the lanes on it left(red) right(blue)
#    return out_img, window_img, filled_lanes_img, left_curverad, right_curverad,center_diff
    return colored_img,left_curverad, right_curverad,pos


In [ ]:
def measure_curvature(left_fit,right_fit):
        ym = 30/720
        xm = 3.7/700

        left_fit = left_fit.copy()
        right_fit = right_fit.copy()
        y_eval = 700 * ym

        # Compute R_curve (radius of curvature)
        left_curveR =  ((1 + (2*left_fit[0] *y_eval + left_fit[1])**2)**1.5)  / np.absolute(2*left_fit[0])
        right_curveR = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        xl = np.dot(left_fit, [700**2, 700, 1])
        xr = np.dot(right_fit, [700**2, 700, 1])
        pos = (1280//2 - (xl+xr)//2)*xm
        return left_curveR, right_curveR, pos 

## region of interest

In [ ]:
def region_of_interest(img):
  h=img.shape[0]
  w=img.shape[1]
  # polygons = np.array([[100,h],[400,570],[600,530],[750,500],[870,530],[1200,h]])
  polygons = np.array([[30,h],[w,h],[w,0],[1300,0],[630,460],[480,500]])
  #  [(100,height),(480,int(height/2)),(900,height)] 3/5 2/3 
  #  [(0,h),(int(0.1*w),int(h* 2/3)),(int(0.8*w),int(h* 2/3)),(int(0.9*w),h)] 
  #  [200,h],[600,700],[700,650],[1020,600],[1200,800],[1400,h]
  #  [100,h],[400,570],[600,530],[750,450],[870,530],[1200,h]
  mask=np.zeros_like(img)
  cv2.fillPoly(mask,pts=[polygons],color=(255,255,255))
  masked_image = cv2.bitwise_and(mask,img)
  return masked_image

## crop

In [ ]:
def crop(img):
  h=img.shape[0]
  w=img.shape[1]
  polygons = np.array([[(130,h),(w,h),(w,0),(130,0)]])#[(100,height),(480,int(height/2)),(900,height)]
  mask=np.zeros_like(img)
  cv2.fillPoly(mask,polygons,255)
  masked_image = cv2.bitwise_and(mask,img)
  return masked_image

## remove_shadow_noise

In [ ]:
def remove_noise(img):
    h=img.shape[0]
    w=img.shape[1]
    half_w=int(w/2)
    polygons = np.array([[(half_w-300,h),(half_w+375,h),(half_w+380,0),(half_w-280,0)]]) 
    mask=np.ones_like(img)
    cv2.fillPoly(mask,polygons,0)
    masked_image = cv2.bitwise_and(mask,img)
    return masked_image


## combinesd thresholds

In [ ]:
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    isX = 1 if orient == 'x' else 0
    sobel = cv2.Sobel(gray, cv2.CV_64F, isX, 1-isX)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel)) 
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel)) 
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return mag_binary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    grad_dir = np.arctan2(abs_sobely, abs_sobelx)
    dir_binary = np.zeros_like(grad_dir)
    dir_binary[(grad_dir >= thresh[0]) & (grad_dir <= thresh[1])] = 1
    return dir_binary

def apply_thresholds(image, ksize=3):
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(10, 100))#(20, 100)
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(10, 100))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    return combined


def apply_color_threshold(image):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    return s_binary


def combine_threshold(s_binary, combined):
    combined_binary = np.zeros_like(combined)
    combined_binary[(s_binary == 1) |(s_binary == 255) | (combined == 1) |(combined == 255)] = 1
    return combined_binary



In [ ]:
def threshold_rel(img, lo, hi):
    vmin = np.min(img)
    vmax = np.max(img)
    
    vlo = vmin + (vmax - vmin) * lo
    vhi = vmin + (vmax - vmin) * hi
    return np.uint8((img >= vlo) & (img <= vhi)) * 255

def threshold_abs(img, lo, hi):
    return np.uint8((img >= lo) & (img <= hi)) * 255

def forward(img):
      """ Take an image and extract all relavant pixels.
      Parameters:
          img (np.array): Input image
      Returns:
          binary (np.array): A binary image represent all positions of relavant pixels.
      """
      hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
      hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
      h_channel = hls[:,:,0]
      l_channel = hls[:,:,1]
      s_channel = hls[:,:,2]
      v_channel = hsv[:,:,2]

      right_lane = threshold_rel(l_channel, 0.8, 1.0)
      right_lane[:,:750] = 0

      left_lane = threshold_abs(h_channel, 20, 30)
      left_lane &= threshold_rel(v_channel, 0.7, 1.0)
      left_lane[:,550:] = 0

      img2 = left_lane | right_lane

      return img2

## cocatenate images

In [ ]:

def combine_images( lane_area_img1, edges, warp,warp_rect,warp_highlight, lane):        
        
        background = np.zeros_like(lane_area_img1)
        large_img_size = (background.shape[1] - int(background.shape[1]/3),int(0.75*background.shape[0])) 
        small_img_size=(int(background.shape[1]/3),int(0.25*background.shape[0]))
        small_img_x_offset=0 
        small_img_y_offset=0
        x=int((0.75*background.shape[0])/2)
        new_image=(int(background.shape[1]/3),x)

        warp_img = cv2.resize(np.dstack((warp, warp, warp))*255,small_img_size)
        edges_image = cv2.resize(np.dstack((edges, edges, edges))*255,small_img_size)
        lane_img = cv2.resize(lane,new_image)
        road = cv2.resize(lane_area_img1,large_img_size)
        warp_rect_img = cv2.resize(warp_rect,small_img_size)
        warp_highlight_img = cv2.resize(warp_highlight,new_image)

        background[0: small_img_size[1], 0: small_img_size[0]] = edges_image
        
        start_offset_y = small_img_y_offset 
        endy = start_offset_y + small_img_size[1]
        if endy > background.shape[0]:
          endy = background.shape[0]
        start_offset_x = 2 * small_img_x_offset + small_img_size[0]
        endx = start_offset_x + small_img_size[0]
        if endx > background.shape[1]:
          endx = background.shape[1]
        background[start_offset_y:endy , start_offset_x: endx] = warp_img
        
        start_offset_y = small_img_y_offset 
        endy = start_offset_y + small_img_size[1]
        if endy > background.shape[0]:
          endy = background.shape[0]
        start_offset_x = 3 * small_img_x_offset + 2 * small_img_size[0]
        endx = start_offset_x + small_img_size[0]
        if endx > background.shape[1]:
          endx = background.shape[1]
        background[start_offset_y: endy, start_offset_x: endx] = warp_rect_img

        start_offset_y = small_img_y_offset 
        endy = x +int(0.25*background.shape[0])
        if endy > background.shape[0]:
          endy = background.shape[0]
        # start_offset_x = 3 * small_img_x_offset + 3 * small_img_size[0]
        # endx = start_offset_x + small_img_size[0]
        if endx > background.shape[1]:
          endx = background.shape[1]
        background[int(0.25*background.shape[0]): endy, background.shape[1] - int(background.shape[1]/3): ] = warp_highlight_img
        background[endy: , background.shape[1] - int(background.shape[1]/3): ] = lane_img
        background[int(0.25*background.shape[0]):  , 0:background.shape[1] - int(background.shape[1]/3)  ] = road
        
        
        return background

## process7_debug

In [ ]:
def process7_debug(input):
 
    # step 1 get edges
    warped=perspective_warp(input)

    after_canny=canny(input)
    # blur = cv2.blur(input,(5,5))
    # gradient_combined = apply_thresholds(blur) #input
    # # Color thresholding
    # s_binary = apply_color_threshold(blur) #input
    # # Combine Gradient and Color thresholding
    # combined_binary = combine_threshold(s_binary, gradient_combined)
    # combined = combine_threshold(after_canny,combined_binary)
    # # step 2 warp canny image

    # after_warp1=perspective_warp(combined)
    # after_warp2 = remove_noise(after_warp1)

    
    after_warp=forward(warped)
    rect = Draw_rectangle(after_warp)
    inv_rect =inv_perspective_warp(rect) 

    # step 3 paint area between lanes

    paint,left_curverad, right_curverad,center_diff= search_around_poly(after_warp)

    # step 4 inverse perspective
    frame =inv_perspective_warp(paint, 
                        dst_size=(input.shape[1],input.shape[0]),
                        src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                        dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]))


    # step 5 add painting to input image
    result = cv2.addWeighted(frame, 0.3, input, 0.7, 0,dtype = cv2.CV_8U) 
    curverad= (left_curverad + right_curverad)/2
    side_pos = 'right'
    if center_diff <= 0:
        side_pos = 'left'
    cv2.putText(result, 'Radius of Curvature='+str(round(curverad,3))+'m ',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    out = combine_images(result, after_canny, after_warp, rect, paint, inv_rect) # after_canny, after_warp,input must be binary

    return result

In [ ]:
# input=plt.imread("/content/drive/MyDrive/test_images/challange_img.jpg")
# out = process7_debug(input)
# plt_images(input,"input",out,"output")
# Input_video_path='/content/drive/MyDrive/test_images/project_video.mp4'
# Output_video_path='video_tharwat.mp4'
# clip1 = VideoFileClip(Input_video_path)
# video_clip = clip1.fl_image(process7_debug) 
# video_clip.write_videofile(Output_video_path, audio=False)

## process7_no_debug

In [ ]:
def process7_debug(input):
 
    # step 1 get edges
    warped=perspective_warp(input)
    after_canny=canny(input)
    after_warp=forward(warped)
    rect = Draw_rectangle(after_warp)
    inv_rect =inv_perspective_warp(rect) 

    # step 3 paint area between lanes

    paint,left_curverad, right_curverad,center_diff= search_around_poly(after_warp)

    # step 4 inverse perspective
    frame =inv_perspective_warp(paint, 
                        dst_size=(input.shape[1],input.shape[0]),
                        src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                        dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]))


    # step 5 add painting to input image
    result = cv2.addWeighted(frame, 0.3, input, 0.7, 0,dtype = cv2.CV_8U) 
    curverad= (left_curverad + right_curverad)/2
    side_pos = 'right'
    if center_diff <= 0:
        side_pos = 'left'
    cv2.putText(result, 'Radius of Curvature='+str(round(curverad,3))+'m ',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    # out = combine_images(result, after_canny, after_warp, rect, paint, inv_rect) # after_canny, after_warp,input must be binary

    return result

## to process a video and save the output


In [ ]:
def create_video(Input_video_path,Output_video_path,mode):
  clip1 = VideoFileClip(Input_video_path)
  if (mode==1):
    video_clip = clip1.fl_image(process7_debug) 
    video_clip.write_videofile(Output_video_path, audio=False)
  else:
    video_clip = clip1.fl_image(process7) 
    video_clip.write_videofile(Output_video_path, audio=False)

  
  

In [34]:
!pip install nbconvert

In [35]:
!jupyter nbconvert --to html last_version.ipynb


[NbConvertApp] WARNING | pattern 'last_version.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
  